In [1]:
import math

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
#start arabic test
import numpy as np
import csv
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.stem.porter import *
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc
import codecs
import sys



import os
from nltk.parse import stanford as SParse
from nltk.tag import stanford as STag
from nltk.tokenize import StanfordSegmenter

from polyglot.text import Text
from rake_nltk import Rake

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

os.environ['STANFORD_MODELS'] = 'stanford-segmenter-2018-10-16/data/;stanford-postagger-full-2018-10-16/models/'
os.environ['STANFORD_PARSER'] = 'stanford-parser-full-2018-10-17'
os.environ['CLASSPATH'] = 'stanford-parser-full-2018-10-17'
os.environ['JAVAHOME'] = 'C:/Program Files/Java/jdk1.8.0_60'

In [4]:
#readind and preparing data ( text , label )
data=pd.read_excel('incident13type.xlsx', header=None) ## read le file
data.columns=['label','text']
print(data[2:3])

   label                                               text
2      4  اقدم المدعو وليد عدنان مرعي على شتم و اهانة ال...


In [7]:
import nltk 
nltk.download('stopwords') 
from nltk.corpus import stopwords
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9ا-ي #+_]')
STOPWORDS = set(stopwords.words('arabic'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = text.replace('x', '')
#    text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text
data['text'] =data['text'].apply(clean_text)
data['text'] =data['text'].str.replace('\d+', '')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Maya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


subData = data['text']
#subData = subData[1:10]
singleTextWithPips = '|'.join(subData)
#print(singleTextWithPips)
f = open("samplePips.txt", "w",  encoding='utf-8')
f.write(singleTextWithPips)
f.close()



segmenter = StanfordSegmenter('stanford-segmenter-2018-10-16/stanford-segmenter-3.9.2.jar')
segmenter.default_config('ar')
textseg = segmenter.segment_file('samplePips.txt')
print(textseg[1:20])

texts=textseg.split('|')
print(texts[1:3])

from nltk.stem import ISRIStemmer
st=ISRIStemmer()
sw=STOPWORDS
collection_new=[]
for  doc in texts:
    new_doc=[]
    for i in doc.split():
        if not i in sw:
            i=st.stem(i)
            i=''.join(i)
            new_doc.append(i)
    collection_new.append(new_doc)
collection=collection_new

collection=texts
print(collection[1])

import re
print(collection[1])
for i in range(len(collection)):
    collection[i] = re.sub(r'\d+', '', str(collection[i]))
print(collection[1])


table=str.maketrans("+=_-<>~/!*%?:;,)&1234567890@$qwertyuioplkjhgfdsamnbvcxz(QWERTYUIOPLKJHGFDSAZXCVBNM",82*" ")
for i in range(len(collection)):
    collection[i] = collection[i].translate(table)
print(collection[1])


In [8]:
#import os, sys
#import pandas as pd
#import numpy as np
#from sklearn.metrics.pairwise import cosine_similarity
#from sklearn.feature_extraction.text import TfidfVectorizer
#from sklearn.model_selection import train_test_split
#from sklearn.naive_bayes import BernoulliNB
#from sklearn.ensemble import RandomForestClassifier
##from sklearn.metrics import roc_curve, auc
#from sklearn import svm
#from sklearn.metrics import classification_report, confusion_matrix
#from sklearn.model_selection import GridSearchCV
from gensim.models import KeyedVectors
#from scipy import spatial
#import matplotlib.pyplot as plt
#from sklearn.neural_network import MLPClassifier
from nltk import bigrams
import nltk
import re
#import spacy
###FUNCTIONS
def avg_feature_vector(sentence, model, num_features, index2word_set):
        words = sentence.split()
        feature_vec = np.zeros((num_features, ), dtype='float32')
        n_words = 0
        for word in words:
            if word in index2word_set:
                n_words += 1
                feature_vec = np.add(feature_vec, model[word])
        if (n_words > 0):
            feature_vec = np.divide(feature_vec, n_words)
        return feature_vec

def convert_word2vec(sentences,num_features):
        vectors_matrix = np.zeros((len(sentences),num_features))
        i = 0
        for sent in sentences:
            temp = avg_feature_vector(sent,word2vec_vectors,num_features,index2word_set)
            vectors_matrix[i,:] = temp
            i = i+1
        return vectors_matrix
def cleaning(doc):
    # Lemmatizes and removes stopwords
    # doc needs to be a spacy Doc object
    #txt = [token for token in doc ]
    words=str(doc).strip().split()
    #words=[st.stem(w) for w in words ]
    return ' '.join(words)
    # Word2Vec uses context words to learn the vector representation of a target word,
    # if a sentence is only one or two words long,
    # the benefit for the training is very small
    #if len(txt) > 2:
      #return ' '.join(txt)
#from gensim.models.phrases import Phrases, Phraser
#data=pd.read_excel('ml2.xlsx', header=None) ## read le file
#data.columns=['text','label']
#def prep(doc):
 #   words=str(doc).strip().split()
  #  return ' '.join(words)
#Xt=Xt.apply(prep)
#sentences = Xt
data=data['text']
#data = data[data['Semantic_Type']!='DISREGARD']
#from gensim.models.phrases import Phrases, Phraser
#from nltk.stem import ISRIStemmer
#st=ISRIStemmer()
brief_cleaning = (re.sub("[^ا-ي']+", ' ', str(row)).lower() for row in data)
txt = [cleaning(doc) for doc in data]
df_clean = pd.DataFrame({'clean': txt})
df_clean = df_clean.dropna().drop_duplicates()
df_clean.shape
sent = [row.split() for row in df_clean['clean']]
sentences = sent
print(sentences[1])
from gensim.models import Word2Vec
word2vec_vectors = Word2Vec(min_count=2,window=5,size=200,sample=6e-5,alpha=0.03,min_alpha=0.0007,negative=20)
word2vec_vectors.build_vocab(sentences)
word2vec_vectors.train(sentences, total_examples=word2vec_vectors.corpus_count, epochs=30, report_delay=1)
word2vec_vectors.init_sims(replace=True)
index2word_set = set(word2vec_vectors.wv.index2word)
###vectorizing the sentences
#vectors_matrix = convert_word2vec(data['Sentence'],200)
#np.random.seed(10)
#labels=data['label']
#train,test,ltrain,ltest =train_test_split(vectors_matrix, labels,test_size=0.2)#,random_state=8)

['التاسعة', 'مسا', 'يوم', 'وفي', 'محلة', 'الطريق', 'الجديدة', 'ارض', 'جلول', 'اوقفت', 'دورية', 'مكتب', 'معلومات', 'بيروت', 'الرابعة', 'المدعو', 'خالد', 'عسكر', 'الاشتباه', 'بترويج', 'المخدرات', 'عثرت', 'بحوزته', 'كمية', 'المخدرات', 'الشكل', 'التالي', 'حبة', 'نوع', 'بنزكسول', 'حبة', 'نوع', 'بنزكسول', 'موضبين', 'داخل', 'ثلاثة', 'اكياس', 'صغيرة', 'حبات', 'ونصف', 'نوع', 'ترامال', 'حبة', 'نوع', 'المخدرة', 'للاعصاب', 'حبة', 'مجهولة', 'الصنف', 'لون', 'اصفر', 'داخل', 'حنجور', 'بالاضافة', 'الى', 'كارت', 'نوع', 'تيليكارت', 'مغلفين']


C:\Users\Maya\Anaconda3\lib\site-packages\gensim\models\base_any2vec.py:743: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  "C extension not loaded, training will be slow. "


In [14]:
print (data[1])

التاسعة مسا يوم    وفي محلة الطريق الجديدة ارض جلول اوقفت دورية مكتب معلومات بيروت الرابعة المدعو خالد عسكر الاشتباه بترويج المخدرات عثرت بحوزته كمية المخدرات الشكل التالي  حبة نوع بنزكسول   حبة نوع بنزكسول موضبين داخل ثلاثة اكياس صغيرة  حبات ونصف نوع ترامال حبة نوع المخدرة للاعصاب حبة مجهولة الصنف لون اصفر داخل حنجور بالاضافة الى  كارت نوع تيليكارت مغلفين


In [9]:
def clean_str(text):
    search = ["أ","إ","آ","ة","_","-","/",".","،"," و "," يا ",'"',"ـ","'","ى","\\",'\n', '\t','&quot;','?','؟','!']
    replace = ["ا","ا","ا","ه"," "," ","","",""," و"," يا","","","","ي","",' ', ' ',' ',' ? ',' ؟ ',' ! ']
    
    #remove tashkeel
    p_tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    text = re.sub(p_tashkeel,"", text)
    
    #remove longation
    p_longation = re.compile(r'(.)\1+')
    subst = r"\1\1"
    text = re.sub(p_longation, subst, text)
    
    text = text.replace('وو', 'و')
    text = text.replace('يي', 'ي')
    text = text.replace('اا', 'ا')
    
    for i in range(0, len(search)):
        text = text.replace(search[i], replace[i])
    
    #trim    
    text = text.strip()

    return text
def get_vec(n_model,dim, token):
    vec = np.zeros(dim)
    is_vec = False
    if token not in n_model.wv:
        _count = 0
        is_vec = True
        for w in token.split("_"):
            if w in n_model.wv:
                _count += 1
                vec += n_model.wv[w]
        if _count > 0:
            vec = vec / _count
    else:
        vec = n_model.wv[token]
    return vec

def calc_vec(pos_tokens, neg_tokens, n_model, dim):
    vec = np.zeros(dim)
    for p in pos_tokens:
        vec += get_vec(n_model,dim,p)
    for n in neg_tokens:
        vec -= get_vec(n_model,dim,n)
    
    return vec   


In [10]:
t_model = word2vec_vectors
pos_tokens=[ clean_str(t.strip()).replace(" ", "_") for t in ['خالد عسكر', 'مخدرات'] if t.strip() != ""]
neg_tokens=[ clean_str(t.strip()).replace(" ", "_") for t in ['سوري'] if t.strip() != ""]

vec = calc_vec(pos_tokens=pos_tokens, neg_tokens=neg_tokens, n_model=t_model, dim=t_model.vector_size)

most_sims = t_model.wv.similar_by_vector(vec, topn=10)
for term, score in most_sims:
    if term not in pos_tokens+neg_tokens:
        print(term, score)

عدد 0.9446272850036621
اقامته 0.942651093006134
انها 0.9370145797729492
منتهية 0.9354233741760254
عشر 0.9325315952301025
الصلاحية 0.9259116649627686
ثلاثة 0.9212719202041626
وسلم 0.9195424318313599
توقيفهم 0.9161526560783386


In [11]:
 word2vec_vectors.wv.vocab

{'الساعة': <gensim.models.keyedvectors.Vocab at 0x27084aa4e80>,
 'تاريخه': <gensim.models.keyedvectors.Vocab at 0x27084c86cf8>,
 'وفي': <gensim.models.keyedvectors.Vocab at 0x27084c86eb8>,
 'محلة': <gensim.models.keyedvectors.Vocab at 0x27084c86d30>,
 'المزرعة': <gensim.models.keyedvectors.Vocab at 0x27084c86f98>,
 'تقاطع': <gensim.models.keyedvectors.Vocab at 0x27084c86fd0>,
 'ابي': <gensim.models.keyedvectors.Vocab at 0x27086741048>,
 'حيدر': <gensim.models.keyedvectors.Vocab at 0x27086741080>,
 'سلام': <gensim.models.keyedvectors.Vocab at 0x270867410b8>,
 'صدمت': <gensim.models.keyedvectors.Vocab at 0x270867410f0>,
 'سيارة': <gensim.models.keyedvectors.Vocab at 0x27086741128>,
 'نوع': <gensim.models.keyedvectors.Vocab at 0x27086741160>,
 'نيسان': <gensim.models.keyedvectors.Vocab at 0x27086741198>,
 'ميكرا': <gensim.models.keyedvectors.Vocab at 0x270867411d0>,
 'لون': <gensim.models.keyedvectors.Vocab at 0x27086741208>,
 'رمادي': <gensim.models.keyedvectors.Vocab at 0x27086741240>,


In [12]:
from itertools import islice
list(islice(word2vec_vectors.wv.vocab, 130, 150))

['اصابة',
 'بشعر',
 'كتفه',
 'الايسر',
 'ادخل',
 'طراد',
 'للمعاجة',
 'المحضر',
 'تعرضت',
 'اميرة',
 'عارف',
 'والدتها',
 'ليلى',
 'تويوتا',
 'اسود',
 'فوزي',
 'بيطار',
 'منى',
 'الحادث',
 'برضة']